## Goes through the transactions and members csvs and looks for members who have not paid or someone paying who is not listed as a member and displays member count for the inspected time frame.

*In the following cell, insert the year, month, and day of month range you want to inspect*

In [ ]:
year = 2020
month = 5
days = (1, 30)

In [ ]:
from datetime import datetime
import pandas as pd

In [ ]:
def str_to_dt(date_string):
    return datetime.strptime(date_string, '%m/%d/%Y')

In [ ]:
#import csv's
members_df = pd.read_csv('/home/data/members.csv')
transactions_df = pd.read_csv('/home/data/transactions.csv')

In [ ]:
# add a more useful date column to transactions_df
transactions_df["datetime"] = transactions_df['Date'].apply(str_to_dt)

In [ ]:
# I only care about paypal payments
# this is future proofing
transactions_df = transactions_df[(transactions_df['Account Name'] == 'PayPal Account')]

In [ ]:
# I only care about last completed month
transactions_df = transactions_df[ (transactions_df['datetime']>pd.Timestamp(year,month,days[0])) & (transactions_df['datetime']<pd.Timestamp(year,month,days[1])) ]

In [ ]:
# I only care about payments (marked as 'income' by pp)
transactions_df = transactions_df[(transactions_df['Category'] == 'Income')]

In [ ]:
transactions_df = transactions_df.reset_index()

In [ ]:
members_list = members_df['Name'].tolist()

In [ ]:
indx = len('Recurring ') # lazy
payers_list = [element[indx:] for element in transactions_df['Description'].tolist()]

In [ ]:
# these members and payers will almost always be cleared (usually by nickname or family plan)
cleared_members = []
# nicknames
cleared_members += ['Nate Swanberg', 'Andy Sheldon', 'Mia Sherman', 'Steve Graves', 'Tom Germon', 'Satar Muzamil']
cleared_payers = ['Nathaniel Swanberg', 'Andrew Sheldon', 'Miamakes', 'Stephen Graves', 'Thomas Germon', 'Art By']
# family members
cleared_members += ['Ashley Keller', "William LeBlanc"]
# kickstarter
cleared_members += ['Gary Skinner', 'John Taylor']
# yearly subscription
cleared_members += ['Tommy Philips']

In [ ]:
members_to_clear = []
for member in members_list:
    if member not in payers_list and member not in cleared_members:
        members_to_clear.append(member)

In [ ]:
payers_to_clear = []
for payer in payers_list:
    if payer not in members_list and payer not in cleared_payers:
        payers_to_clear.append(payer)

In [ ]:
yes = ['yes', 'Yes', 'y', 'Y']
no = ['no', 'No', 'n', 'N']
for name in members_to_clear:
    clear = '!'
    while clear not in yes and clear not in no:
        clear = input('clear '+ name+'? ')
    if clear in yes:
        cleared_members.append(name)
    

In [ ]:
for payer in payers_to_clear:
    clear = '!'
    while clear not in yes and clear not in no:
        clear = input('clear '+ payer+'? ')
    if clear in yes:
        cleared_payers.append(payer)
    

In [ ]:
uncleared_members = [member for member in members_to_clear if member not in cleared_members]
uncleared_payers = [payer for payer in payers_to_clear if payer not in cleared_payers]

In [ ]:
if len(uncleared_members) > 0:
    print('I couldn\'t clear non-payment for the following (they may have cancelled): ')
    for name in uncleared_members:
        print('- '+name)
        
if len(uncleared_payers) > 0:
    print('the following payers are not listed as members:')
    for payer in uncleared_payers:
        print(payer)


In [ ]:
print("number of confirmed members:")
print(len(members_list) - len(uncleared_members))